In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [ ]:
# Check if CUDA is available and print the status
if torch.cuda.is_available():
    print("CUDA is available, using GPU")
    device = torch.device("cuda")
else:
    print("CUDA not available, using CPU")
    device = torch.device("cpu")

In [ ]:
# Transform for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and Dataloaders
train_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Smaller batch size for MobileNet
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Number of classes: {len(train_dataset.classes)}')
print(f'Class to index mapping: {train_dataset.class_to_idx}')


In [ ]:
# Load MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 12)  # Modify last layer for 12 classes
model.to(device)

# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if i % 50 == 49:  # Print more frequently due to smaller batch size
            print(f'Batch {i+1}, Loss: {running_loss/50:.3f}, Accuracy: {100.*correct/total:.2f}%')
            running_loss = 0.0
    
    epoch_accuracy = 100. * correct / total
    return epoch_accuracy, running_loss / len(dataloader)

In [ ]:
def evaluate(dataloader, model, loss_fn):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(dataloader)
    test_accuracy = 100. * correct / total
    return test_loss, test_accuracy

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_accuracy, train_loss = train_epoch(train_loader, model, loss_fn, optimizer)
    test_loss, test_accuracy = evaluate(test_loader, model, loss_fn)
    
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    print('-' * 40)
    
    # Learning rate scheduling
    scheduler.step(test_loss)

print('Training completed.')